Notebook to replicate our results for data set 2 which uses a RNN for condition procession

In [ ]:
import copy
import torch.nn as nn
import torch
from FrEIA.framework import *
from FrEIA.modules import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
device = torch.device("cuda:0")
pad = lambda x: nn.utils.rnn.pad_sequence(x, batch_first = True).to(device)
pack = lambda x, y: nn.utils.rnn.pack_padded_sequence(x, y, batch_first=True, enforce_sorted=False)

In [ ]:
def load_data():
    """
    Helper function to read the data which is saved in multiple .csv files
    converts data to pytorch tensors
    """
    all_data = []
    all_params = []
    all_length = []
    i = 0
    j = 0
    while(True):
        name_dat = f"./data_mobility{j}/dat{i}.csv"
        name_param = f"./params_mobility{j}/params{i}.csv"
        try:
            df_dat = pd.read_csv(name_dat)
            df_param = pd.read_csv(name_param)
            param = df_param.to_numpy()[0,[0,1,10]]
            data = df_dat.iloc[:,2].to_numpy().reshape(-1,7)[:,[0,1,2,3]]
            if len(data[np.isnan(data)]) == 0:
                all_data.append(torch.from_numpy(data).to(torch.float32).to(device))
                all_params.append(param)
                all_length.append(len(data))
            else:
                print(i)
            i += 1
        except Exception as e:
            i = 0
            j += 1
            print("Loaded",len(all_data),"Entries")
        if j>8:
            print("not so much data")
            break
        if len(all_data)%1000 == 0:
            print("Loaded",len(all_data),"Entries")
    all_params = torch.from_numpy(np.array(all_params)).to(torch.float32).to(device)
    return all_data, all_params, all_length

"""
# save the data in a torch file which is much faster to load then the .csvs

data, params, length = load_data()
data_dict = {"data": data,
            "params": params,
            "length": length}

torch.save(data_dict, "data_second.save")
"""

In [ ]:
data_dict = torch.load("data_second.save", map_location = device)

In [ ]:
data = data_dict["data"][:290000]
padded_data = pad(data)
params = data_dict["params"][:290000]
length = data_dict["length"][:290000]
test_dat = data_dict["data"][290000:]
test_params = data_dict["params"][290000:]
test_length = data_dict["length"][290000:]

In [ ]:
class preprocessor():
    """
    Used to save initial means and range of the parameters and
    applies the preprocessing
    
    """
    def __init__(self, params):
        self.means = params.mean(dim = 0)
        self.max = torch.abs(params-self.means).max(dim = 0)[0]+1e-5
 
    def __call__(self, params, fwd = True):
        if fwd:
            return torch.arctanh((params - self.means)/self.max)
        else:
            return torch.tanh(params) * self.max + self.means

prepper = preprocessor(params)
prepped = prepper(params, fwd = True)

In [ ]:
class Decoder(nn.Module):
    """
    Decoder base class for use in an autoencoder
    N: amount of linear layers
    hidden_size: width of the hidden layers
    out_size: output dim of the decoder, should match the rnns input dim
    lr: learning rate
    """
    def __init__(self, N, inp_size, hidden_size, out_size, lr = 1e-3):
        super(Decoder, self).__init__()
        self.inp_size = inp_size
        self.linear = get_linear_subnet(N, inp_size, hidden_size, out_size)
        self.params_trainable = list(filter(
                lambda p: p.requires_grad, self.linear.parameters())) 
        n_trainable = sum(p.numel() for p in self.params_trainable)
        print(f"Number of Decoder parameters: {n_trainable}", flush=True)        
        self.optimizer = torch.optim.AdamW(
                self.params_trainable,
                lr = lr,
                betas =[0.9, 0.999],
                eps = 1e-6,
                weight_decay = 0
            )
        self.scheduler =  torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer,
                verbose = True
            )        
        
    def forward(self, x):
        return self.linear(x)

class AE():
    """
    Autoencoder class used to pretrain the RNN
    as decoder a simple fcnn is used
    rnn: object of type RNN which is trained
    N: amount of linear layers for the decoder
    hidden_size: width of the hidden decoder layers
    dim: output dim of the decoder, should match the rnns input dim
    """
    def __init__(self, rnn, N = 6, hidden_size = 256, dim = 4):
        self.dim = dim
        self.encoder = rnn
        self.decoder = Decoder(N, rnn.get_dim(), hidden_size, 149 * self.dim)
    def train(self, epochs, x, lengths, batch_size = 2048):
        loss_curve = []
        metrics_curve = []
        zeros = torch.zeros((batch_size,149,self.dim), dtype = torch.float32, device = device)
        for epoch in range(epochs):
            epoch_index = np.random.permutation(len(x))
            epoch_losses = 0
            for i in range(len(x)//batch_size):
                xsamps = x[epoch_index][i*batch_size:(i+1)*batch_size]
                length = lengths[epoch_index][i*batch_size:(i+1)*batch_size]
                
                packed_xsamps = pack(xsamps, length)
                
                self.decoder.optimizer.zero_grad(set_to_none=True)
                self.encoder.optimizer.zero_grad(set_to_none=True)
                encoded = self.encoder(packed_xsamps)
                decoded = self.decoder(encoded).view(-1,149,self.dim)
                
                difference = (torch.where(xsamps != 0, decoded, zeros)-xsamps)**2
                loss = torch.mean(difference)
                if loss < 1e30:
                    loss.backward()
                else:
                    print(f"loss is {loss}")
                    return
                self.encoder.optimizer.step()
                self.decoder.optimizer.step()
                epoch_losses += loss.item()/(len(x)//batch_size)
            loss_curve.append(epoch_losses)
            self.encoder.scheduler.step(epoch_losses)
            self.decoder.scheduler.step(epoch_losses)
            print("Epoch:", epoch + 1)
            print("Loss:", epoch_losses)
        self.decoder.optimizer.zero_grad(set_to_none=True)
        self.encoder.optimizer.zero_grad(set_to_none=True)
        plt.plot(np.arange(len(loss_curve)),np.array(loss_curve))      
        
    def inference(self, x):
        with torch.no_grad():
            decoded = self.decoder(self.encoder(x.view(1,-1,self.dim))).view(149,self.dim).cpu().numpy()
        xnump = x.cpu().numpy()
        fig, ax = plt.subplots(7, figsize = (8,16))
        names = ["Susceptible", "Infected", "Immune", "Dead", "Asymptomatic", "Hospitalized", "Severe"]
        for i in range(7):
            ax[i].plot(np.arange(len(xnump)), xnump[:,i], label = f"Truth")
            ax[i].plot(np.arange(149), decoded[:,i], label = f"Fake")
            ax[i].set_ylabel(names[i])
            
            ax[i].legend()
            ax[i].set_ylim([0,1])
        ax[3].set_xlabel("Time in days")
        plt.savefig("rnn.pdf")

In [ ]:
def get_linear_subnet(N, inp_size, hidden_size, out_size):
    """
    Helper function to get linear network with ReLU activation
    N: amount of layers, not including the final linear output layer
    inp_size: dimension of the input
    hidden_size: width of the N-1 hidden layers
    out_size: dimension of the output
    """
    layer_list = []
    layer_list.append(nn.Linear(inp_size, hidden_size))
    layer_list.append(nn.ReLU())
    for i in range(N-1):
        layer_list.append(nn.Linear(hidden_size, hidden_size))
        layer_list.append(nn.ReLU())
    layer_list.append(nn.Linear(hidden_size, out_size))
    return nn.Sequential(*layer_list).to(device)

class RNN(nn.Module):
    """"
    RNN baseclass, used as conditioning network for the cINN. Implemented as an LSTM
    inp_size: dimension of the input timeseries
    hidden_size: parallel channels used by the rnns
    numm_rnns: amount of lstms
    lr: learning rate
    bi: wether to use bidirectional lstms or not
    """
    def __init__(self, inp_size, hidden_size = 5, num_rnns = 6, lr = 1e-3, bi = False):
        super(RNN, self).__init__()
        self.inp_size = inp_size
        self.rnn = nn.LSTM(inp_size, hidden_size, num_rnns, batch_first = True, bidirectional = bi).to(device)
        self.params_trainable = list(filter(
                lambda p: p.requires_grad, self.rnn.parameters())) 
        n_trainable = sum(p.numel() for p in self.params_trainable)
        print(f"Number of RNN parameters: {n_trainable}", flush=True)        
        self.optimizer = torch.optim.AdamW(
                self.params_trainable,
                lr = lr,
                betas =[0.9, 0.99],
                eps = 1e-6,
                weight_decay = 0
            )
        self.scheduler =  torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer,
                verbose = True
            )        
        
    def forward(self, x):
        full, (last, cn) = self.rnn(x)
        return torch.swapaxes(last, 0, 1).reshape(last.shape[1], -1)
    
    def get_dim(self):
        """
        passes dummy variable through network to get the outputs shape
        """
        return self.forward(torch.randn(1,100,self.inp_size, device = device)).shape[1]
        
    
class cINN(nn.Module):
    """
    cINN baseclass, using cubic spline blocks.
    inp_size: dimension of the input
    cond_size: dimension of the conditions
    num_blocks: amount of coupling blocks used
    sub_layers: amount of linear layers in the subnetworks
    sub_width: width of the subnetworks
    lr: learning rate, scheduler used is reduce_on_plateau
    """
    def __init__(self, inp_size, cond_size, num_blocks = 10, sub_layers = 3, sub_width = 256, lr = 2e-4):
        super(cINN, self).__init__()
        constructor_fct = lambda x_in, x_out: get_linear_subnet(sub_layers, 
                                                                x_in,
                                                                x_in,
                                                                x_out)

        block_kwargs = {
                        "num_bins": 120,
                        "subnet_constructor": constructor_fct,
                        "bounds_init": 5,
                        "permute_soft": False
                           }
        inp_size = (inp_size,)        
        nodes = [InputNode(*inp_size, name='inp')]
        cond_node = ConditionNode(*(cond_size,))
        for i in range(num_blocks):
            nodes.append(Node(
                    [nodes[-1].out0],
                    CubicSplineBlock,
                    block_kwargs,
                    conditions = cond_node,
                    name = f"block_{i}",
                    
                ))
        nodes.append(OutputNode([nodes[-1].out0], name='out'))
        nodes.append(cond_node)
        self.model = GraphINN(nodes, verbose=False).to(device)
        self.params_trainable = list(filter(
                lambda p: p.requires_grad, self.model.parameters()))
        n_trainable = sum(p.numel() for p in self.params_trainable)
        print(f"Number of cINN parameters: {n_trainable}", flush=True)
        
        self.optimizer = torch.optim.AdamW(
                self.params_trainable,
                lr = lr,
                betas =[0.9, 0.99],
                eps = 1e-6,
                weight_decay = 0
            )
        self.scheduler =  torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer,
                verbose = True
            )
    def forward(self, x, cond = None, rev = False):
        return self.model(x, c = cond, rev = rev)


class Estimator():
    """
    Wrapper for training cINN and RNN at the same time, also used for inference
    """
    def __init__(self, rnn, cinn):
        self.cinn = cinn
        self.rnn = rnn

    def metrik(self, true, data, zeros):
        """
        Used to measure the distance between truth and generated parameters
        Better interpretable than the Loss function
        """
        with torch.no_grad():
            cond = self.rnn(data)
            output, _ = self.cinn(zeros, cond, rev = True)
            output = torch.abs(prepper(output, fwd = False) - prepper(true, fwd = False))
            output = torch.mean(output, dim = 0).cpu().numpy()
        print("Metric:", output)
        return output
    
    def train(self, epochs, xtrain, ytrain, lengths, batch_size, train_rnn = True):
        """
        Trains the cINN and shows loss and metric plots
        epochs: amount of epochs to train
        xtrain: training parameters
        ytrain: corresponding time series, need to be zero padded
        lengths: real lengths of the time series, used for packing them again
        batch_size: Batch size used for the training
        train_rnn: Enable weight updates for the RNN
        """
        loss_curve = []
        metrics_curve = []
        zeros = torch.zeros(batch_size,5).to(device)
        for epoch in range(epochs):
            epoch_index = np.random.permutation(len(xtrain))
            epoch_losses = 0
            for i in range(len(xtrain)//batch_size):
                ysamps = ytrain[epoch_index][i*batch_size:(i+1)*batch_size]
                xsamps = xtrain[epoch_index][i*batch_size:(i+1)*batch_size]
                length = lengths[epoch_index][i*batch_size:(i+1)*batch_size]
                
                packed_ysamps = pack(ysamps, length)
                
                self.cinn.optimizer.zero_grad(set_to_none=True)
                if train_rnn:
                    self.rnn.optimizer.zero_grad(set_to_none=True)
                
                cond = self.rnn(packed_ysamps)

                gauss, jac = self.cinn(xsamps, cond)
                loss = torch.mean(gauss**2/2) - torch.mean(jac)/gauss.shape[1]
                    
                loss.backward()
                self.cinn.optimizer.step()
                if train_rnn:
                    self.rnn.optimizer.step()
                epoch_losses += loss.item()/(len(xtrain)//batch_size)
            loss_curve.append(epoch_losses)
            
            self.cinn.scheduler.step(epoch_losses)
            if train_rnn:
                self.rnn.scheduler.step(epoch_losses)
            print("Epoch:", epoch + 1)
            print("Loss:", epoch_losses)
            metrics_curve.append(self.metrik(xsamps,packed_ysamps, zeros))
        plt.plot(np.arange(len(loss_curve)),np.array(loss_curve), label = "Loss")
        plt.ylabel("Loss")
        plt.xlabel("Epoch")
        plt.show()
        metrics_curve = np.array(metrics_curve)
        for i in range(metrics_curve.shape[1]):
            plt.plot(np.arange(len(loss_curve)), metrics_curve[:,i], label = f"Metric $x_{i}$")
        plt.xlabel("Epoch")
        plt.ylabel("Metric")
        
    def inference(self, data_point, true_param, twod = False):
        """
        Predicts a parameter distribution for given time series
        Overlays the true parameter to see how well the network performs
        twod: Addionally plot 2d correlations
        """
        outputs = []
        with torch.no_grad():
            for i in range(100):
                gauss = torch.randn(1000,5).to(device)
                cond = self.rnn(data_point.repeat(1000,1,1).to(device))
                output, _ = self.cinn(gauss, cond, rev = True)
                outputs.append(output)
        output = torch.cat(outputs, dim = 0)
        output = prepper(output, fwd = False).detach().cpu()
        names = ["Initial infected", "Initial immune", "Mobility susceptible", "Mobility infected", "Mobility immune"]
        fig, axis = plt.subplots(3,2, figsize = (10,12))
        for i in range(len(names)):
            ax = axis[int((i-i%2)/2),i%2]
            ax.hist(output.numpy()[:,i], bins = 100, density = True, label = "Generated")
            ax.axvline(true_param[i], color = "r", label = "Truth")
            ax.set_xlabel(names[i], fontsize = 12)
            ax.legend()
            if i%2 == 0:
                ax.set_ylabel("Normalized", fontsize = 12)
        plt.show()
        if twod:
            fig, axis = plt.subplots(2,2, figsize = (10,12))
            for i in range(3):
                ax = axis[int((i-i%2)/2),i%2]
                hist, xedges, yedges = np.histogram2d(output.numpy()[:,2+i], output.numpy()[:,2+(i+1)%3], bins=[100,100], density=True)
                mapp = ax.pcolormesh(xedges, yedges, hist.T, rasterized=True)
                ax.set_xlabel(names[2+i])
                ax.set_ylabel(names[2+(i+1)%3])
            fig.colorbar(mapp, cax = axis[1,1])

In [ ]:
rnn = RNN(7, hidden_size = 10, num_rnns = 6, lr = 1e-3, bi = False)
ae = AE(rnn, dim = 7)
ae.train(10, padded_data, np.array(length))

In [ ]:
ae.inference(test_dat[np.random.randint(1000)])

In [ ]:
cinn = cINN(5, rnn.get_dim())
network = Estimator(rnn, cinn)
network.train(10, prepped, padded_data, np.array(length), batch_size = 1024, train_rnn = False)

In [ ]:
network.train(200, prepped, padded_data, np.array(length), batch_size = 1024, train_rnn = True)

In [ ]:
i = np.random.randint(len(test_dat))
network.inference(test_dat[i], test_params[i])

In [ ]:
i = np.random.randint(len(test_dat))
network.inference(test_dat[i], test_params[i], twod = True)